In [1]:
import os, sys
import geopandas as gpd
import pandas as pd
# import rasterio as rio
# import numpy as np
# from shapely.geometry import Point
# import skimage.graph as graph
from geopy.geocoders import GeoNames, Nominatim

In [2]:
# sys.path.append('/home/wb514197/Repos/gostrocks/src') # gostrocks is used for some basic raster operations (clip and standardize)
# sys.path.append('/home/wb514197/Repos/GOSTNets_Raster/src') # gostnets_raster has functions to work with friction surface
# sys.path.append('/home/wb514197/Repos/GOSTnets') # it also depends on gostnets for some reason
# sys.path.append('/home/wb514197/Repos/INFRA_SAP') # only used to save some raster results
# sys.path.append('/home/wb514197/Repos/HospitalAccessibility/src') # only used to save some raster results

In [3]:
# import GOSTRocks.rasterMisc as rMisc
# import GOSTNetsRaster.market_access as ma
# from infrasap import aggregator

In [4]:
iso3 = 'LBR'

In [5]:
input_dir = "/home/public/Data/PROJECTS/Health" #
out_folder = os.path.join(input_dir, "output", iso3)
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

### Destinations

In [6]:
lbr_master = pd.read_excel(os.path.join(input_dir, "from_tashrik", "master lists", "Liberia.xlsx"))

In [7]:
lbr_master['status'] = lbr_master['Status'].apply(lambda x: x.lower())
lbr_master['hf_type'] = lbr_master['HF Type'].apply(lambda x: x.lower())

In [8]:
lbr = lbr_master.loc[lbr_master.Status!="non-functional"].copy()

In [9]:
lbr_missing = lbr.loc[lbr.Lat.isna()].copy()

In [10]:
lbr_non_missing = lbr.loc[~(lbr.Lat.isna())].copy()

In [11]:
# row = lbr_missing.iloc[0]
# idx = row.name

In [14]:
list(lbr_missing.columns)

['County',
 'District',
 'health district',
 'Location',
 'New HF ID',
 'UUID Validated (DHIS2)',
 'HF_name validated (DHIS2)',
 'UUID',
 'HF Name',
 'Population (2011)',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'F_Type by EPHS classification',
 'HF Type',
 'Owership',
 'Status',
 'Established by',
 'Locality Type',
 'Lat',
 'Long',
 'Q008_Manag_Authority',
 'facility_contact_person',
 'facility_contact_position',
 'facility_person_contact',
 'number available?',
 'facility_person_contact2',
 '# of inpatient beds',
 '# of dedicated maternity inpatient beds',
 'Vehicle or Functional Ambulance',
 'Access to ambulance',
 'Water source',
 'On premise water available',
 'Function Refrigerator for Vaccine storage',
 'Functional Incinerator',
 'Functional Pit',
 'Means of waste disposal',
 'Functional Computer',
 'Functional Triage',
 'Dedicated Isolation room',
 'Functional landline for communication',
 'Cellular Network',
 'Access to phone',
 'Functional Short Wave Radio',
 'Access 

### 1 Check if location already exists in master list

In [1]:
lbr_non_missing.loc[lbr_non_missing.Location==row.Location,['Lat', 'Long']]
# lbr.loc[lbr["health district"]=="Klay",['Lat', 'Long']]

NameError: name 'lbr_non_missing' is not defined

In [16]:
lbr_copy = lbr_non_missing.loc[lbr_non_missing.Location==row.Location, ['Lat', 'Long']]

In [17]:
len(lbr_copy)

0

### 2 Geocode

In [18]:
geolocator = Nominatim(user_agent="health-equity")
# from geopy.extra.rate_limiter import RateLimiter
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [19]:
# copied from docs
# import pandas as pd
# df = pd.DataFrame({'name': ['paris', 'berlin', 'london']})

# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="specify_your_app_name_here")

# from geopy.extra.rate_limiter import RateLimiter
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# df['location'] = df['name'].apply(geocode)

# df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [20]:
lbr_missing.Location = lbr_missing.Location.astype(str)

In [21]:
i = 1
for idx, row in lbr_missing.iterrows():
    print(f"Geocoding {idx}, {i} of {len(lbr_missing)}")
    # Check if there ia another entry with location
    lbr_copy = lbr_non_missing.loc[lbr_non_missing.Location==row.Location,['Lat', 'Long']]
    if len(lbr_copy)>0:
        res = lbr_copy.iloc[0]
        lbr_missing.loc[idx, "Long"] = res.Long
        lbr_missing.loc[idx, "Lat"] = res.Lat
        print("\tSuccess for Another facility with same Location")
        lbr_missing.loc[idx, "geocoding_method"] = "Another facility with same Location"
    else:
        if "Community" in row['Location']:
            lbr_missing.loc[idx, "Location"] = row['Location'].replace(" Community", "")
        try:
            res = geolocator.geocode(row['Location'], country_codes='LBR')
            lbr_missing.loc[idx, "Long"] = res.longitude
            lbr_missing.loc[idx, "Lat"] = res.latitude
            print("\tSuccess for Location")
            lbr_missing.loc[idx, "geocoding_method"] = "Location name"
        except:
            try:
                res = geolocator.geocode(row['health district'], country_codes='LBR')
                lbr_missing.loc[idx, "Long"] = res.longitude
                lbr_missing.loc[idx, "Lat"] = res.latitude
                print("\tSuccess for health district")
                lbr_missing.loc[idx, "geocoding_method"] = "Health District name"
            except:
                print('\tFailed')
                lbr_missing.loc[idx, "geocoding_method"] = "Failed"
    i += 1

Geocoding 3, 1 of 72
	Success for health district
Geocoding 88, 2 of 72
	Success for Location
Geocoding 115, 3 of 72
	Success for health district
Geocoding 157, 4 of 72
	Success for health district
Geocoding 159, 5 of 72
	Success for Location
Geocoding 170, 6 of 72
	Success for Location
Geocoding 173, 7 of 72
	Success for Another facility with same Location
Geocoding 175, 8 of 72
	Success for Another facility with same Location
Geocoding 176, 9 of 72
	Success for health district
Geocoding 178, 10 of 72
	Success for Another facility with same Location
Geocoding 181, 11 of 72
	Success for Another facility with same Location
Geocoding 182, 12 of 72
	Success for Another facility with same Location
Geocoding 184, 13 of 72
	Success for Location
Geocoding 188, 14 of 72
	Success for health district
Geocoding 193, 15 of 72
	Success for Another facility with same Location
Geocoding 198, 16 of 72
	Success for Location
Geocoding 240, 17 of 72
	Success for health district
Geocoding 244, 18 of 72
	S

In [22]:
lbr_missing.geocoding_method.value_counts()

Location name                          31
Health District                        27
Another facility with same Location    14
Name: geocoding_method, dtype: int64

In [23]:
lbr_missing.to_csv(os.path.join(out_folder, "health_geocoded.csv"))

In [24]:
# geoms = [Point(xy) for xy in zip(lbr.Long, lbr.Lat)]
# lbr_geo = gpd.GeoDataFrame(lbr, crs='EPSG:4326', geometry=geoms)